In [33]:
import numpy as np
import matplotlib.pyplot as plt
import math
from ipywidgets import interact
import matplotlib.patches as matpatch
from descartes.patch import PolygonPatch
from shapely.ops import cascaded_union, polygonize
from matplotlib.patches import Circle
from matplotlib.patches import Rectangle
from scipy.integrate import quad, dblquad
from scipy import interpolate
import shapely.geometry as sg
import descartes
from scipy import optimize 
from mpl_toolkits.mplot3d import Axes3D
#import sobol_seq
%matplotlib notebook

# Алгоритм Монте-Карло для нахождения площади сложной фигуры

<img src="2rpr.jpg">

https://www.researchgate.net/publication/245038755_Workspace_optimization_and_kinematic_performance_evaluation_of_2DOF_parallel_mechanisms

### Робот 2-RPR
Робот состоит из двух штанг переменной длинны и известно расстояние между основаниями штанг

Задача: найти площадь рабочей области робота и нарисовать её

In [2]:
def check_circ_lit(x, y, x0, y0, r):#проверяем лежит ли точка вне малой окр-ти
    """
    Check point if it is outside lower limit of leg (what is the same if these points outside the smallest circles)
    
    Parameters
    ----------
    x, y : float, the coordinates of points
    x0, y0 : float, the coordinates of the circle's center
    r : float, radius of the circle
    Returns
    -------
    Boolean massive of all points which are outside the smallest circles
    
    """
    return (((x - x0)**2 + (y - y0)**2) >= r**2)

In [3]:
def check_circ_big(x, y, x0, y0, r):#проверяем лежит ли точка вне малой окр-ти
    """
    Check points if it is inside the higher limit of leg (what is the same if these points inside the biggest circles)
    
    Parameters
    ----------
    x, y : float, the coordinates of points
    x0, y0 : float, the coordinates of the circle's center
    r : float, radius of the circle
    Returns
    -------
    Boolean massive of all points which are inside the biggest circles
    
    return ((x - x0)**2 + (y - y0)**2 <= r**2)
    
    """
    return ((x - x0)**2 + (y - y0)**2 <= r**2)

In [55]:
def fun(x):
    return [x[0]**2+x[1]**2-l1**2,
            x[1]**2+(d-x[0])**2-l2**2]

In [52]:
l1_l = 3
l2_l = 3
l1_h = 25
l2_h = 25

In [59]:
interact(visual_cond_numb, d=(3,15))

interactive(children=(IntSlider(value=9, description='d', max=15, min=3), Output()), _dom_classes=('widget-int…

<function __main__.visual_cond_numb(d)>

In [48]:
def visual_cond_numb(d):
    L1 = np.linspace(l1_l, l1_h, l1_h-l1_l)
    L2 = np.linspace(l2_l, l2_h, l2_h-l2_l)
    L1, L2 = np.meshgrid(L1, L2)
    K_mat = np.zeros([l1_h-l1_l, l2_h-l2_l])
    T_mat = np.zeros([l1_h-l1_l, l2_h-l2_l])
    S_mat = np.zeros([l1_h-l1_l, l2_h-l2_l])
    M_mat = np.zeros([l1_h-l1_l, l2_h-l2_l])
    for i in range(l1_l, l1_h):
        for j in range(l2_l, l2_h):
            K_mat[i-l1_l,j-l2_l] = cond_numb(i, j, d)[0]
            T_mat[i-l1_l,j-l2_l] = cond_numb(i, j, d)[1]
            if cond_numb(i, j, d)[2]!=0:
                S_mat[i-l1_l,j-l2_l] = 1/cond_numb(i, j, d)[2]
            else:
                S_mat[i-l1_l,j-l2_l] = cond_numb(i, j, d)[2]
            M_mat[i-l1_l,j-l2_l] = cond_numb(i, j, d)[3]
    fig = plt.figure(figsize=(10, 10))
    
    ax1 = fig.add_subplot(221, projection='3d')
    ax1.set_ylabel('l2')
    ax1.set_xlabel('l1')
    ax1.set_title('The condition number of Jacobian matrix')
    ax1.set_zlabel('k')
    surf = ax1.plot_surface(L1, L2, K_mat, cmap='rainbow',
                           linewidth=0.1, antialiased=False)
    #cbar = fig.colorbar(surf)
    ax1.grid()
    print('Min condition number of Jacobian matrix',np.min(K_mat[K_mat>=1]))
    
    ax2 = fig.add_subplot(222, projection='3d')
    ax2.set_ylabel('l2')
    ax2.set_xlabel('l1')
    ax2.set_title('Transmission quality index')
    ax2.set_zlabel('T')
    surf = ax2.plot_surface(L1, L2, T_mat, cmap='rainbow',
                           linewidth=0.1, antialiased=False)
    #cbar = ax2.colorbar(surf)
    ax2.grid()
    print('Max transmission quality ',np.max(T_mat))
    
    ax3 = fig.add_subplot(223, projection='3d')
    ax3.set_ylabel('l2')
    ax3.set_xlabel('l1')
    ax3.set_title('Stiffness index')
    ax3.set_zlabel('S')
    surf = ax3.plot_surface(L1, L2, S_mat, cmap='rainbow',
                           linewidth=0.1, antialiased=False)
    #cbar = ax2.colorbar(surf)
    ax3.grid()
    print('Max stiffness index ',np.max(S_mat))
    
    ax4 = fig.add_subplot(224, projection='3d')
    ax4.set_ylabel('l2')
    ax4.set_xlabel('l1')
    ax4.set_title('Manipulability index')
    ax4.set_zlabel('M')
    surf = ax4.plot_surface(L1, L2, M_mat, cmap='rainbow',
                           linewidth=0.1, antialiased=False)
    #cbar = ax2.colorbar(surf)
    ax4.grid()
    print('Max manipulability index ',np.max(M_mat))

In [58]:
def cond_numb(l1, l2, d):
    #sol = optimize.root(fun, [0, 0], method='hybr')
    #x = sol.x
    k = 0
    T = 0
    S = 0
    M = 0
    J = np.array([[0 , 0],
                [0, 0]])
    if (l1<=l2+d) and (l2<=l1+d) and (d<=l1+l2):
        x1 = (d**2+l1**2-l2**2)/(2*d)
        x2 = math.sqrt(l1**2-x1**2)
        #print(x1,x2)
        J_teta = np.array([[l1, 0],
                       [0, l2]])
        J_x = np.array([[x1, x2],
                   [x1-d, x2]])
        #if np.linalg.det(J_x) == 0:
        #    print('Сингулярность первого типа')
        #    print(l1,l2)
        #if np.linalg.det(J_teta) == 0:
        #    print('Сингулярность второго типа типа')
        #    print(l1,l2)
        if (np.linalg.det(J_x) != 0) and (np.linalg.det(J_teta) != 0):
            J = np.dot(-np.linalg.inv(J_teta), J_x)
            #print(J)
            k = np.linalg.norm(J)*np.linalg.norm(np.linalg.inv(J))
            T = (np.linalg.norm(np.eye(2))**2)/(np.linalg.norm(J)*np.linalg.norm(np.linalg.inv(J)))
            S = np.linalg.norm(np.linalg.inv(np.dot(J,J.T)))*np.linalg.norm(np.dot(J,J.T))
            M = np.linalg.det(J)
            #print(k, T, S, M)
    return(k, T, S, M)

### Алгоритм Монте-Карло
1) Ограничим нашу рабочую область прямоугольником с известными сторонами

2) Заполним прямоугольник случайным образом $N$-точками

3) Площадь искомой области будет вычисляться по формуле: $S=\dfrac{(b-a)\cdot (c-d) \cdot  K}{N}$, где $a$ и $b$ - левая и правая границы интегрирования, $с$ и $d$ - верхняя и нижняя границы интегрирования, $K$ - кол-во точек, которые попали в искомую область

Все точки рабочей области должны удоволетворять условиям, что они вне малых окружностей и внутри больших окружностей

In [4]:
def Monte_Carlo(l1_l, l2_l, l1_h, l2_h, d, p, seed=1234):
    """ 
    Compute the area of 2-RPR robot's workspace with Monte Carlo method
    
    Parameters
    ----------
    l1_l, l2_l : float, the lowest limit of legs
    l1_h, l2_h : float, the highest limit of legs
    d : float, length between legs of the robot
    p : int, amount of points
    seed : int, seed for rng, default = 1234
    Returns
    -------
    area: float , area of 2-RPR robot's workspace
    Xl, Xh : float, X-axis limits for MC rectangle
    Yl, Yh : float, Y-axis limits for MC rectangle
    
    """
    prng = np.random.RandomState(seed)
    Xp = (-l2_h**2 + l1_h**2 + d**2)/(2*d)
    Yp = math.sqrt(l1_h**2 - Xp**2)
    if l1_h <= l2_h:
        Yh = l1_h
    else:
        Yh = l2_h
    Xl = d - l2_h
    Xh = l1_h+abs(d - l2_h)
    Yl = 0
    X = prng.uniform(Xl, Xh, size=p)
    Y = prng.uniform(Yl, Yh, size=p)
    c = 0
    c = sum((check_circ_lit(X, Y, 0, 0, l1_l)) & (check_circ_lit(X, Y, d, 0, l2_l)) &
            (check_circ_big(X, Y, d, 0, l2_h)) & (check_circ_big(X, Y, 0, 0, l1_h)))
    area = ((Xh-Xl)*Yh*c/p)
    return area, Xl, Xh, Yl, Yh

In [5]:
def Ravnomern_net(l1_l, l2_l, l1_h, l2_h, d, p, seed=1234):
    """ 
    Compute the area of 2-RPR robot's workspace with Monte Carlo method
    
    Parameters
    ----------
    l1_l, l2_l : float, the lowest limit of legs
    l1_h, l2_h : float, the highest limit of legs
    d : float, length between legs of the robot
    p : int, amount of points
    seed : int, seed for rng, default = 1234
    Returns
    -------
    area: float , area of 2-RPR robot's workspace
    Xl, Xh : float, X-axis limits for MC rectangle
    Yl, Yh : float, Y-axis limits for MC rectangle
    
    """
    prng = np.random.RandomState(seed)
    Xp = (-l2_h**2 + l1_h**2 + d**2)/(2*d)
    Yp = math.sqrt(l1_h**2 - Xp**2)
    if l1_h <= l2_h:
        Yh = l1_h
    else:
        Yh = l2_h
    Xl = d - l2_h
    Xh = l1_h+abs(d - l2_h)
    Yl = 0
    X1 = np.linspace(Xl, Xh+Xl, p)
    Y1 = np.linspace(Yl, Yh, p)
    X, Y = np.meshgrid(X1, Y1)
    X = X.ravel()
    Y = Y.ravel()
    c = 0
    c = sum((check_circ_lit(X, Y, 0, 0, l1_l)) & (check_circ_lit(X, Y, d, 0, l2_l)) &
            (check_circ_big(X, Y, d, 0, l2_h)) & (check_circ_big(X, Y, 0, 0, l1_h)))
    print(Xh,Xl,Yh,c,p)
    area = ((Xh-Xl)*Yh*c/p**2)
    return area, Xl, Xh, Yl, Yh

In [6]:
def draw_and_compute(l1_l, l2_l, l1_h, l2_h, d):
    """ 
    Draw the workspace of 2-RPR robot, computing its area with shapely and MC-algorithm 
    
    Parameters
    ----------
    l1_l, l2_l : float, the lowest limit of legs
    l1_h, l2_h : float, the highest limit of legs
    d : float, length between legs of the robot
    Returns
    -------
    
    """
    p=100000
    Ax=[]
    Ay=[]
    if (l1_l<l1_h) and (l2_l<l2_h):
        area,Xl,Xh,Yl,Yh=Monte_Carlo(l1_l,l2_l,l1_h,l2_h,d,p)
        area_ravn = Ravnomern_net(l1_l,l2_l,l1_h,l2_h,d,100)[0]
        X1 = np.linspace(Xl, Xh+Xl, 10)
        Y1 = np.linspace(Yl, Yh, 10)
        X, Y = np.meshgrid(X1, Y1)
        l1_l+=0.00000000000001
        l2_l+=0.00000000000001
        fig, ax = plt.subplots()
        x_min, y_min, x_max, y_max=-l1_h-1,-l1_h-1,d+l2_h+1,l2_h+1
        ax.set_xlim([x_min, x_max])
        ax.set_ylim([y_min, y_max])
        a = sg.Point(0,0).buffer(l1_l)# задаём оркужности через shapely, чтобы построить пересечение колец и найти площадь пересечения
        b = sg.Point(0,0).buffer(l1_h)
        c = sg.Point(d,0).buffer(l2_l)
        e = sg.Point(d,0).buffer(l2_h)
        ab=b.difference(a)# строим кольца из большой и малой окружности
        cd=e.difference(c)
        middle = ab.intersection(cd)
        print('Площадь через алгоритм Монте-Карло =',area)
        print('Площадь через равномерную сетку =',area_ravn)
        print('Приблизительное значение площади, посчитанная через shapely =',middle.area/2)
        circle1 = Circle((0, 0), radius=l1_l, fill=False, color='r')# задаём окружности через matplotlib
        circle2 = Circle((0, 0), radius=l1_h, fill=False, color='r')
        circle3 = Circle((d, 0), radius=l2_l, fill=False, color='b')
        circle4 = Circle((d, 0), radius=l2_h, fill=False, color='b')
        rect1=Rectangle([Xl,Yl],Xh,Yh, fill=False, color='g', linewidth=2.0)
        ax.add_patch(descartes.PolygonPatch(middle, fc='b', ec='k', alpha=0.2))# отрисовываем
        ax.add_patch(circle1)
        ax.add_patch(circle2)
        ax.add_patch(circle3)
        ax.add_patch(circle4)
        ax.add_patch(rect1)
        ax.scatter(X,Y)
        ax.grid()
        ax.axes.set_aspect('equal')
    else:
        print('Неверные данные')

### Рабочая область 2-RPR
Рабочая область робота представляет собой пересечение колец, образованных попарно окружностями малых и больших длин каждой штанги, так же стоит отметить, что эта область симметрична относително оси Х и нам достаточно будет найти площадь верхней(или нижней) полуплоскости. 

In [78]:
interact(draw_and_compute, l1_l=(3,15), l2_l=(3,15), l1_h=(5,20), l2_h=(5,20), d=(3,15))

interactive(children=(IntSlider(value=9, description='l1_l', max=15, min=3), IntSlider(value=9, description='l…

<function __main__.draw_and_compute(l1_l, l2_l, l1_h, l2_h, d)>

In [123]:
J_mat[:, J_mat>1]

IndexError: too many indices for array

In [118]:
J_mat

array([[0.        , 0.        , 0.        , 0.        , 0.        ,
        2.41897263, 2.03520158, 2.02288695, 2.2453656 , 2.95481765,
        0.        , 0.        ],
       [0.        , 0.        , 0.        , 0.        , 2.56205046,
        2.10544542, 2.00173837, 2.04124145, 2.2043095 , 2.56205046,
        3.47100814, 0.        ],
       [0.        , 0.        , 0.        , 2.63180678, 2.15410109,
        2.01917863, 2.00445931, 2.06559112, 2.20176211, 2.44452933,
        2.89533012, 3.97613368],
       [0.        , 0.        , 2.63180678, 2.17088168, 2.03267122,
        2.        , 2.02524724, 2.09656967, 2.21739157, 2.40535118,
        2.70426503, 3.23316151],
       [0.        , 2.56205046, 2.15410109, 2.03267122, 2.00041662,
        2.01361026, 2.05921942, 2.13504205, 2.2453656 , 2.40153208,
        2.62694372, 2.97251318],
       [2.41897263, 2.10544542, 2.01917863, 2.        , 2.01361026,
        2.04964037, 2.10544542, 2.1821789 , 2.28402953, 2.41897263,
        2.60127046,